In [64]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matsim
import importlib
import pollutants
import matplotlib.pyplot as plt
from lxml import etree
import travel_events_anls as travel_anls
import shipment_events_anls as shipment_anls

In [151]:
importlib.reload(pollutants)
importlib.reload(travel_anls)


<module 'travel_events_anls' from '/Volumes/External/gitProj/matsim-xp/src/main/python/freight_emission/travel_events_anls.py'>

# Path

In [154]:
root_path = r'../../../../data/intermediate/test/freightEmissions/'
iter_kw = "0"
basic_scenario_outputs = 'scenarioBasic/iter'+iter_kw+'/outputs/'
van_scenario_outputs = 'scenarioVan/iter'+iter_kw+'/outputs/'
cargo_bike_scenario_outputs = 'scenarioCB/iter' + iter_kw +'/outputs/'

scenario_kw = cargo_bike_scenario_outputs


# Read Network

In [3]:
network = matsim.read_network(
    root_path + 'GemeenteLeuvenWithHbefaType.xml.gz'
)
network_gdf = network.as_geo()
network_gdf

,length,freespeed,capacity,permlanes,oneway,modes,link_id,from_node,to_node,geometry
0,7.221720,13.888889,1277.777778,2.0,1,bike,100006229_1-1085376891_0,453075429,453075431,"LINESTRING (175094.462 171579.032, 175087.718 ..."
1,7.924752,13.888889,1277.777778,2.0,1,bike,100006230_1-1085376884_0,1472593946,1156261854,"LINESTRING (175094.956 171547.998, 175102.682 ..."
2,8.518773,13.888889,1277.777778,2.0,1,bike,100006231_0-1085376890_0,1156261856,453075428,"LINESTRING (175114.241 171571.774, 175106.042 ..."
3,50.128561,33.333333,2666.666667,3.5,1,"bike,car",100010104_0,1156293852,1156293879,"LINESTRING (174065.630 177001.873, 174056.230 ..."
4,14.524426,33.333333,2666.666667,3.5,1,"bike,car",100010104_1,1156293879,1156293875,"LINESTRING (174056.230 177051.111, 174070.620 ..."
...,...,...,...,...,...,...,...,...,...,...
103748,183.145612,33.333333,2666.666667,3.5,1,car,splitLink_9904390_5_2,splitNode_9904390_5_1,19792038,"LINESTRING (175632.401 172452.198, 175505.340 ..."
103749,102.305105,33.333333,2666.666667,3.5,1,car,splitLink_9904410_0_1,80702464,splitNode_9904410_0_1,"LINESTRING (176002.946 172889.690, 176056.498 ..."
103750,102.305105,33.333333,2666.666667,3.5,1,car,splitLink_9904410_0_2,splitNode_9904410_0_1,100399122,"LINESTRING (176056.498 172976.855, 176110.050 ..."
103751,113.694402,13.888889,1277.777778,2.0,1,bike,splitLink_99346726_1_1,1149134729,splitNode_99346726_1_1,"LINESTRING (170932.314 173317.687, 171004.317 ..."


In [4]:
nodes = network.nodes
nodes

,x,y,node_id
0,175270.308015,173512.798102,1000630419
1,175160.942490,173472.188651,1000630421
2,175238.142000,173547.294517,1000630459
3,175338.370411,173481.813410,1000630462
4,175180.404081,173465.163020,1000630464
...,...,...,...
59045,174985.241824,172934.604012,splitNode_9904064_1_1
59046,174985.241824,172934.604012,splitNode_9904064_r_1_1
59047,175632.401153,172452.198438,splitNode_9904390_5_1
59048,176056.498044,172976.854738,splitNode_9904410_0_1


# Read carrier plans

In [5]:
# Load the XML file
file_path = root_path + scenario_kw + 'output_carriers.xml.gz'
tree = etree.parse(file_path)
root = tree.getroot()

# Namespace to handle the XML structure
namespace = {"ns": "http://www.matsim.org/files/dtd"}

In [6]:
# Function to extract shipments
def extract_shipments(root, namespace):
    shipments = []
    for carrier in root.findall("ns:carrier", namespaces=namespace):
        carrier_id = carrier.get("id")
        for shipment in carrier.find("ns:shipments", namespaces=namespace).findall("ns:shipment", namespaces=namespace):
            shipments.append({
                "carrier_id": carrier_id,
                "shipment_id": shipment.get("id"),
                "from": shipment.get("from"),
                "to": shipment.get("to"),
                "size": shipment.get("size"),
                "start_pickup": shipment.get("startPickup"),
                "end_pickup": shipment.get("endPickup"),
                "start_delivery": shipment.get("startDelivery"),
                "end_delivery": shipment.get("endDelivery"),
                "pickup_service_time": shipment.get("pickupServiceTime"),
                "delivery_service_time": shipment.get("deliveryServiceTime")
            })
    return pd.DataFrame(shipments)

In [7]:
# Function to extract vehicles
def extract_vehicles(root, namespace):
    vehicles = []
    for carrier in root.findall("ns:carrier", namespaces=namespace):
        carrier_id = carrier.get("id")
        capabilities = carrier.find("ns:capabilities", namespaces=namespace)
        if capabilities is not None:
            for vehicle in capabilities.find("ns:vehicles", namespaces=namespace).findall("ns:vehicle", namespaces=namespace):
                vehicles.append({
                    "carrier_id": carrier_id,
                    "vehicle_id": vehicle.get("id"),
                    "depot_link_id": vehicle.get("depotLinkId"),
                    "type_id": vehicle.get("typeId"),
                    "earliest_start": vehicle.get("earliestStart"),
                    "latest_end": vehicle.get("latestEnd")
                })
    return pd.DataFrame(vehicles)

In [8]:
def extract_paired_tours_from_file(root, namespace):
    tours = []
    for carrier in root.findall("ns:carrier", namespaces=namespace):
        carrier_id = carrier.get("id")
        plans = carrier.find("ns:plans", namespaces=namespace)
        if plans is not None:
            for plan in plans.findall("ns:plan", namespaces=namespace):
                for tour in plan.findall("ns:tour", namespaces=namespace):
                    tour_id = tour.get("tourId")
                    vehicle_id = tour.get("vehicleId")
                    
                    # Pairing `act` and `leg`
                    acts = tour.findall("ns:act", namespaces=namespace)
                    legs = tour.findall("ns:leg", namespaces=namespace)
                    
                    # Ensure we pair correctly by iterating over both
                    for act, leg in zip(acts, legs):
                        tours.append({
                            "carrier_id": carrier_id,
                            "plan_score": plan.get("score"),
                            "plan_selected": plan.get("selected"),
                            "tour_id": tour_id,
                            "vehicle_id": vehicle_id,
                            "activity_type": act.get("type"),
                            "shipment_id": act.get("shipmentId"),
                            "activity_end_time": act.get("end_time"),
                            "leg_dep_time": leg.get("expected_dep_time"),
                            "leg_transp_time": leg.get("expected_transp_time"),
                            "route": leg.find("ns:route", namespaces=namespace).text
                        })
    tours_df = pd.DataFrame(tours)
    tours_df['route'] = tours_df['route'].str.split(' ')
    return tours_df.query("plan_selected=='true'")

In [9]:
shipments_df = extract_shipments(root, namespace)
shipments_df

,carrier_id,shipment_id,from,to,size,start_pickup,end_pickup,start_delivery,end_delivery,pickup_service_time,delivery_service_time
0,carrier_0,shipment_77,333784188_r_3,284664272_r_7,13,05:00:00,05:50:00,06:00:00,08:00:00,00:00:00,00:01:30
1,carrier_0,shipment_78,333784188_r_3,3343971_0-3343971_1-3343971_2-3343971_3,9,17:00:00,17:50:00,18:00:00,20:00:00,00:00:00,00:01:30
2,carrier_0,shipment_79,333784188_r_3,795726691_r_14,22,17:00:00,17:50:00,18:00:00,20:00:00,00:00:00,00:01:30
3,carrier_0,shipment_73,333784188_r_3,374339593_0-374339593_1,18,17:06:00,17:56:00,18:06:00,20:06:00,00:00:00,00:01:30
4,carrier_0,shipment_74,333784188_r_3,3869832_1-3869832_2,11,05:00:00,05:50:00,06:00:00,08:00:00,00:00:00,00:01:30
...,...,...,...,...,...,...,...,...,...,...,...
195,carrier_0,shipment_134,333784188_r_3,814322393_r_0,9,17:06:00,17:56:00,18:06:00,20:06:00,00:00:00,00:01:30
196,carrier_0,shipment_133,333784188_r_3,814577199_r_0,17,17:06:00,17:56:00,18:06:00,20:06:00,00:00:00,00:01:30
197,carrier_0,shipment_132,27566523_11,893949581_0,17,17:00:00,17:50:00,18:00:00,20:00:00,00:00:00,00:01:30
198,carrier_0,shipment_131,333784188_r_3,295116813_0-295116813_1,12,17:06:00,17:56:00,18:06:00,20:06:00,00:00:00,00:01:30


In [10]:
vehicles_df = extract_vehicles(root, namespace)
vehicles_df

,carrier_id,vehicle_id,depot_link_id,type_id,earliest_start,latest_end
0,carrier_0,carrier0_Van_0,27566523_11,Van,00:00:00,596523:14:07
1,carrier_0,carrier0_Van_1,333784188_r_3,Van,00:00:00,596523:14:07


In [11]:
tour_df = extract_paired_tours_from_file(root, namespace)
tour_df

,carrier_id,plan_score,plan_selected,tour_id,vehicle_id,activity_type,shipment_id,activity_end_time,leg_dep_time,leg_transp_time,route
804,carrier_0,-2880.8889062270546,true,1,carrier0_Van_1,start,None,00:00:00,00:00:00,00:00:00,None
805,carrier_0,-2880.8889062270546,true,1,carrier0_Van_1,pickup,shipment_195,None,05:06:00,00:00:00,None
806,carrier_0,-2880.8889062270546,true,1,carrier0_Van_1,pickup,shipment_5,None,05:06:00,00:00:00,None
807,carrier_0,-2880.8889062270546,true,1,carrier0_Van_1,pickup,shipment_183,None,05:06:00,00:00:00,None
808,carrier_0,-2880.8889062270546,true,1,carrier0_Van_1,pickup,shipment_58,None,05:06:00,00:00:00,None
...,...,...,...,...,...,...,...,...,...,...,...
1201,carrier_0,-2880.8889062270546,true,2,carrier0_Van_1,delivery,shipment_162,None,19:50:18,00:00:06,[892269110_0-892269110_1-892269110_2-892269110...
1202,carrier_0,-2880.8889062270546,true,2,carrier0_Van_1,delivery,shipment_102,None,19:51:54,00:05:21,"[892269122_0, 3343957_2, 3343957_3, 3343957_4,..."
1203,carrier_0,-2880.8889062270546,true,2,carrier0_Van_1,delivery,shipment_134,None,19:58:46,00:00:30,"[814322393_2, 608715520_0, 174338458_0, 174338..."
1204,carrier_0,-2880.8889062270546,true,2,carrier0_Van_1,delivery,shipment_79,None,20:00:46,00:00:44,"[795726691_r_15-795726691_r_16, 795726691_r_17..."


# Events anls

In [12]:
def parse_matsim_events(file_path):

    # Read the XML file
    tree = etree.parse(file_path)
    root = tree.getroot()

    # Get the data of all events
    events_data = []
    for event in root.findall('event'):
        event_data = event.attrib
        events_data.append(event_data)

    # Convert the data to a DataFrame
    events_df = pd.DataFrame(events_data)

    events_df['time'] = pd.to_numeric(events_df['time'], errors='coerce')
    events_df = events_df.dropna(subset=['time'])
    # If the value of person is not type str while the value of vehicle is not, replace the value of person with the value of vehicle
    events_df['person'] = events_df.apply(lambda x: x['vehicle'] if not isinstance(x['person'], str) else x['person'], axis=1)
    events_df['person'] = events_df.apply(lambda x: x['vehicleId'] if not isinstance(x['person'], str) else x['person'], axis=1)
    # events_df = events_df.sort_values(by=['time'])

    # Store the events per vehicle
    vehicle_events = {}
    for vehicle_id, group in events_df.groupby('person'):
        vehicle_events[vehicle_id] = group

    return events_df, vehicle_events

In [144]:
events_df, vehicle_events = parse_matsim_events(root_path + scenario_kw + 'output_events.xml.gz')

In [145]:
events_df

,BC_exhaust,Benzene,CH4,CO,CO2_TOTAL,CO2_rep,CO2e,FC,FC_MJ,HC,...,networkMode,person,pickupDuration,relativePosition,shipmentId,time,tourId,type,vehicle,vehicleId
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_2,NaN,NaN,NaN,0.0,NaN,actend,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_2,NaN,NaN,NaN,0.0,NaN,departure,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_2,NaN,NaN,NaN,0.0,NaN,PersonEntersVehicle,freight_carrier_0_veh_carrier0_Van_1_2,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_1,NaN,NaN,NaN,0.0,NaN,actend,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_1,NaN,NaN,NaN,0.0,NaN,departure,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_1,NaN,NaN,NaN,72573.0,NaN,PersonLeavesVehicle,freight_carrier_0_veh_carrier0_Van_1_1,NaN
20334,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_1,NaN,NaN,NaN,72573.0,NaN,arrival,NaN,NaN
20335,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_1,NaN,NaN,NaN,72573.0,NaN,actstart,NaN,NaN
20336,NaN,1.496732349818508E-6,5.986929399274036E-5,0.01084822357956593,4.022294889019806,3.847879595373764,NaN,1.30394983229438,0.05524995425073463,1.4967323236287023E-4,...,NaN,freight_carrier_0_veh_carrier0_Van_1_1,NaN,NaN,NaN,72573.0,NaN,warmEmissionEvent,NaN,freight_carrier_0_veh_carrier0_Van_1_1


In [147]:
events_df.query("vehicle == 'freight_carrier_0_veh_carrier0_Van_1_1'")

,BC_exhaust,Benzene,CH4,CO,CO2_TOTAL,CO2_rep,CO2e,FC,FC_MJ,HC,...,networkMode,person,pickupDuration,relativePosition,shipmentId,time,tourId,type,vehicle,vehicleId
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_1,NaN,NaN,NaN,0.0,NaN,PersonEntersVehicle,freight_carrier_0_veh_carrier0_Van_1_1,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,car,freight_carrier_0_veh_carrier0_Van_1_1,NaN,1.0,NaN,0.0,NaN,vehicle enters traffic,freight_carrier_0_veh_carrier0_Van_1_1,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,car,freight_carrier_0_veh_carrier0_Van_1_1,NaN,1.0,NaN,0.0,NaN,vehicle leaves traffic,freight_carrier_0_veh_carrier0_Van_1_1,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_1,NaN,NaN,NaN,0.0,NaN,PersonLeavesVehicle,freight_carrier_0_veh_carrier0_Van_1_1,NaN
14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_1,NaN,NaN,NaN,0.0,NaN,PersonEntersVehicle,freight_carrier_0_veh_carrier0_Van_1_1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_1,NaN,NaN,NaN,72571.0,NaN,left link,freight_carrier_0_veh_carrier0_Van_1_1,NaN
20330,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_1,NaN,NaN,NaN,72571.0,NaN,entered link,freight_carrier_0_veh_carrier0_Van_1_1,NaN
20332,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,car,freight_carrier_0_veh_carrier0_Van_1_1,NaN,1.0,NaN,72573.0,NaN,vehicle leaves traffic,freight_carrier_0_veh_carrier0_Van_1_1,NaN
20333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_1,NaN,NaN,NaN,72573.0,NaN,PersonLeavesVehicle,freight_carrier_0_veh_carrier0_Van_1_1,NaN


In [148]:
test_v_events = vehicle_events.get('freight_carrier_0_veh_carrier0_Van_1_1')
test_v_events

,BC_exhaust,Benzene,CH4,CO,CO2_TOTAL,CO2_rep,CO2e,FC,FC_MJ,HC,...,networkMode,person,pickupDuration,relativePosition,shipmentId,time,tourId,type,vehicle,vehicleId
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_1,NaN,NaN,NaN,0.0,NaN,actend,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_1,NaN,NaN,NaN,0.0,NaN,departure,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_1,NaN,NaN,NaN,0.0,NaN,PersonEntersVehicle,freight_carrier_0_veh_carrier0_Van_1_1,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,car,freight_carrier_0_veh_carrier0_Van_1_1,NaN,1.0,NaN,0.0,NaN,vehicle enters traffic,freight_carrier_0_veh_carrier0_Van_1_1,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,car,freight_carrier_0_veh_carrier0_Van_1_1,NaN,1.0,NaN,0.0,NaN,vehicle leaves traffic,freight_carrier_0_veh_carrier0_Van_1_1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_1,NaN,NaN,NaN,72573.0,NaN,PersonLeavesVehicle,freight_carrier_0_veh_carrier0_Van_1_1,NaN
20334,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_1,NaN,NaN,NaN,72573.0,NaN,arrival,NaN,NaN
20335,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_1,NaN,NaN,NaN,72573.0,NaN,actstart,NaN,NaN
20336,NaN,1.496732349818508E-6,5.986929399274036E-5,0.01084822357956593,4.022294889019806,3.847879595373764,NaN,1.30394983229438,0.05524995425073463,1.4967323236287023E-4,...,NaN,freight_carrier_0_veh_carrier0_Van_1_1,NaN,NaN,NaN,72573.0,NaN,warmEmissionEvent,NaN,freight_carrier_0_veh_carrier0_Van_1_1


In [150]:
simple_test_v = test_v_events.query("type == 'Freight shipment delivered ends' or type == 'Freight shipment pickup starts'")
simple_test_v

,BC_exhaust,Benzene,CH4,CO,CO2_TOTAL,CO2_rep,CO2e,FC,FC_MJ,HC,...,networkMode,person,pickupDuration,relativePosition,shipmentId,time,tourId,type,vehicle,vehicleId
26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_1,0.0,NaN,shipment_195,0.0,NaN,Freight shipment pickup starts,freight_carrier_0_veh_carrier0_Van_1_1,NaN
50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_1,0.0,NaN,shipment_5,1.0,NaN,Freight shipment pickup starts,freight_carrier_0_veh_carrier0_Van_1_1,NaN
70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_1,0.0,NaN,shipment_183,2.0,NaN,Freight shipment pickup starts,freight_carrier_0_veh_carrier0_Van_1_1,NaN
94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_1,0.0,NaN,shipment_58,3.0,NaN,Freight shipment pickup starts,freight_carrier_0_veh_carrier0_Van_1_1,NaN
114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_1,0.0,NaN,shipment_24,4.0,NaN,Freight shipment pickup starts,freight_carrier_0_veh_carrier0_Van_1_1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_1,NaN,NaN,shipment_35,71861.0,NaN,Freight shipment delivered ends,freight_carrier_0_veh_carrier0_Van_1_1,NaN
19527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_1,NaN,NaN,shipment_26,71981.0,NaN,Freight shipment delivered ends,freight_carrier_0_veh_carrier0_Van_1_1,NaN
19663,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_1,NaN,NaN,shipment_33,72148.0,NaN,Freight shipment delivered ends,freight_carrier_0_veh_carrier0_Van_1_1,NaN
19731,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_1,NaN,NaN,shipment_111,72244.0,NaN,Freight shipment delivered ends,freight_carrier_0_veh_carrier0_Van_1_1,NaN


In [149]:
travel_anls.identify_shipment_event_seg(test_v_events)

      BC_exhaust Benzene  CH4   CO CO2_TOTAL CO2_rep CO2e   FC FC_MJ   HC  \
2948         NaN     NaN  NaN  NaN       NaN     NaN  NaN  NaN   NaN  NaN   
2967         NaN     NaN  NaN  NaN       NaN     NaN  NaN  NaN   NaN  NaN   
3157         NaN     NaN  NaN  NaN       NaN     NaN  NaN  NaN   NaN  NaN   
3467         NaN     NaN  NaN  NaN       NaN     NaN  NaN  NaN   NaN  NaN   
3519         NaN     NaN  NaN  NaN       NaN     NaN  NaN  NaN   NaN  NaN   
...          ...     ...  ...  ...       ...     ...  ...  ...   ...  ...   
19420        NaN     NaN  NaN  NaN       NaN     NaN  NaN  NaN   NaN  NaN   
19527        NaN     NaN  NaN  NaN       NaN     NaN  NaN  NaN   NaN  NaN   
19663        NaN     NaN  NaN  NaN       NaN     NaN  NaN  NaN   NaN  NaN   
19731        NaN     NaN  NaN  NaN       NaN     NaN  NaN  NaN   NaN  NaN   
19872        NaN     NaN  NaN  NaN       NaN     NaN  NaN  NaN   NaN  NaN   

       ... pickupDuration relativePosition    shipmentId     time tourId  \

/Volumes/External/gitProj/matsim-xp/src/main/python/freight_emission/travel_events_anls.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['time_'] = sub_df['time'].shift(-1)
/Volumes/External/gitProj/matsim-xp/src/main/python/freight_emission/travel_events_anls.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['time_diff'] = sub_df['time_'] - sub_df['time']
/Volumes/External/gitProj/matsim-xp/src/main/python/freight_emission/travel_events_anls.py:31: SettingWithCopyWarning: 
A value is t

([28865.0], [29030.0], 72363.0)

In [ ]:
def identify_shipment_event_seg(v_event: pd.DataFrame, time_threshold: int = 2 * 3600):
    sub_df = v_event.query("type=='Freight shipment delivered ends'")
    # Copy the time column to a new column and move forward by one row
    sub_df['time_'] = sub_df['time'].shift(-1)
    # Calculate the time difference between the two columns
    sub_df['time_diff'] = sub_df['time_'] - sub_df['time']
    # Identify the index of the rows where the time difference is greater than the threshold
    idx = sub_df[sub_df['time_diff'] > time_threshold].index
    # Remain 
    return sub_df

In [ ]:
def clip_events_last_shipment_delivered(vehicle_event: pd.DataFrame):
    last_shipment_delivered = vehicle_event.query("type=='Freight shipment delivered ends'").tail(1)
    last_shipment_delivered_idx = last_shipment_delivered.index[0]
    return vehicle_event.loc[:last_shipment_delivered_idx]

In [36]:
clipped_test_v_events = clip_events_last_shipment_delivered(test_v_events)
clipped_test_v_events

,BC_exhaust,Benzene,CH4,CO,CO2_TOTAL,CO2_rep,CO2e,FC,FC_MJ,HC,...,networkMode,person,pickupDuration,relativePosition,shipmentId,time,tourId,type,vehicle,vehicleId
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_2,NaN,NaN,NaN,0.0,NaN,actend,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_2,NaN,NaN,NaN,0.0,NaN,departure,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_2,NaN,NaN,NaN,0.0,NaN,PersonEntersVehicle,freight_carrier_0_veh_carrier0_Van_1_2,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,car,freight_carrier_0_veh_carrier0_Van_1_2,NaN,1.0,NaN,0.0,NaN,vehicle enters traffic,freight_carrier_0_veh_carrier0_Van_1_2,NaN
15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,car,freight_carrier_0_veh_carrier0_Van_1_2,NaN,1.0,NaN,0.0,NaN,vehicle leaves traffic,freight_carrier_0_veh_carrier0_Van_1_2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19726,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_2,NaN,NaN,shipment_20,72205.0,NaN,Freight shipment delivered starts,freight_carrier_0_veh_carrier0_Van_1_2,NaN
19777,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_2,NaN,NaN,NaN,72296.0,NaN,actend,NaN,NaN
19778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_2,NaN,NaN,NaN,72296.0,NaN,departure,NaN,NaN
19779,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_2,NaN,NaN,NaN,72296.0,NaN,PersonEntersVehicle,freight_carrier_0_veh_carrier0_Van_1_2,NaN


In [37]:
clipped_test_v_travel_events = clipped_test_v_events.query("type == 'entered link' or type == 'left link'")
clipped_test_v_travel_events

,BC_exhaust,Benzene,CH4,CO,CO2_TOTAL,CO2_rep,CO2e,FC,FC_MJ,HC,...,networkMode,person,pickupDuration,relativePosition,shipmentId,time,tourId,type,vehicle,vehicleId
494,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_2,NaN,NaN,NaN,23.0,NaN,left link,freight_carrier_0_veh_carrier0_Van_1_2,NaN
495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_2,NaN,NaN,NaN,23.0,NaN,entered link,freight_carrier_0_veh_carrier0_Van_1_2,NaN
518,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_2,NaN,NaN,NaN,25.0,NaN,left link,freight_carrier_0_veh_carrier0_Van_1_2,NaN
519,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_2,NaN,NaN,NaN,25.0,NaN,entered link,freight_carrier_0_veh_carrier0_Van_1_2,NaN
547,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_2,NaN,NaN,NaN,28.0,NaN,left link,freight_carrier_0_veh_carrier0_Van_1_2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19713,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_2,NaN,NaN,NaN,72186.0,NaN,entered link,freight_carrier_0_veh_carrier0_Van_1_2,NaN
19715,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_2,NaN,NaN,NaN,72193.0,NaN,left link,freight_carrier_0_veh_carrier0_Van_1_2,NaN
19716,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_2,NaN,NaN,NaN,72193.0,NaN,entered link,freight_carrier_0_veh_carrier0_Van_1_2,NaN
19718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,freight_carrier_0_veh_carrier0_Van_1_2,NaN,NaN,NaN,72199.0,NaN,left link,freight_carrier_0_veh_carrier0_Van_1_2,NaN


## Using travel events anls

In [157]:
travel_events = travel_anls.read_travel_events(root_path + scenario_kw, 
                                               #True
                                               )
travel_events

,type,link,time,vehicle
0,left link,333784188_r_3,4.0,freight_carrier_0_veh_carrier0_CB_1_7
1,entered link,326736776_r_5,4.0,freight_carrier_0_veh_carrier0_CB_1_7
2,left link,333784188_r_3,5.0,freight_carrier_0_veh_carrier0_CB_1_12
3,entered link,326736776_r_5,5.0,freight_carrier_0_veh_carrier0_CB_1_12
4,left link,333784188_r_3,6.0,freight_carrier_0_veh_carrier0_CB_1_1
...,...,...,...,...
27503,entered link,326736776_4,66942.0,freight_carrier_0_veh_carrier0_CB_1_7
27504,left link,326736776_4,66944.0,freight_carrier_0_veh_carrier0_CB_1_7
27505,entered link,333784188_0,66944.0,freight_carrier_0_veh_carrier0_CB_1_7
27506,left link,333784188_0,66948.0,freight_carrier_0_veh_carrier0_CB_1_7


In [142]:
travel_events_chains = travel_anls.derive_vehicle_travel_chain(travel_events)
travel_events_chains

({'freight_carrier_0_veh_carrier0_Van_1_2': ['333784188_r_3',
   '326736776_r_5',
   '326736776_r_6',
   '326736776_r_7',
   '326736776_r_8',
   '326736776_r_9',
   '333580518_r_0',
   '333580518_r_1',
   '333580518_r_2',
   '333580518_r_3',
   '333580518_r_4',
   '333580518_r_5',
   '333580518_r_6',
   '333580518_r_7',
   '333580518_r_8',
   '333580518_r_9',
   '333580518_r_10',
   '333580518_r_11',
   '333580518_r_12',
   '333580518_r_13-333580518_r_14',
   '333580518_r_15',
   '333580518_r_16',
   '608754690_0-112917238_0',
   '112917238_1',
   '112917238_2',
   '112917238_3',
   '112917238_4',
   '112917238_5',
   '112917238_6',
   '112917238_7',
   '112917238_8',
   '112917238_9',
   '112917238_10',
   '112917238_11-112917238_12',
   '112917238_13',
   '502317796_0',
   '502317796_1',
   '502317796_2',
   '502317796_3',
   '502317796_4',
   '502317796_5',
   '810580948_0',
   '810580948_1',
   '810580948_2',
   '502317795_0',
   '502317795_1',
   '502317795_2',
   '502317795_3',
 

In [158]:
sum(list(travel_anls.derive_all_vkt_as_dict(travel_events, network_gdf)[0].values()))

250618.56870790463

In [133]:
travel_anls.derive_all_vkt_as_dict(travel_events, network_gdf)

({'freight_carrier_0_veh_carrier0_CB_1_7': 24339.012579260463,
  'freight_carrier_0_veh_carrier0_CB_1_12': 22200.314569716054,
  'freight_carrier_0_veh_carrier0_CB_1_1': 21271.876409180375,
  'freight_carrier_0_veh_carrier0_CB_0_10': 20305.112274195737,
  'freight_carrier_0_veh_carrier0_CB_0_13': 17094.565127114365,
  'freight_carrier_0_veh_carrier0_CB_0_6': 21620.76252621826,
  'freight_carrier_0_veh_carrier0_CB_1_2': 16826.78212326378,
  'freight_carrier_0_veh_carrier0_CB_1_4': 12770.183950139928,
  'freight_carrier_0_veh_carrier0_CB_0_9': 21541.16211974822,
  'freight_carrier_0_veh_carrier0_CB_0_3': 15128.866195232575,
  'freight_carrier_0_veh_carrier0_CB_0_5': 23417.29934623425,
  'freight_carrier_0_veh_carrier0_CB_0_11': 18852.585562167147,
  'freight_carrier_0_veh_carrier0_CB_1_8': 15250.045925433478},
 {'freight_carrier_0_veh_carrier0_CB_1_7': 5523.0,
  'freight_carrier_0_veh_carrier0_CB_1_12': 5424.0,
  'freight_carrier_0_veh_carrier0_CB_1_1': 5045.0,
  'freight_carrier_0_veh_c

In [134]:
travel_anls.derive_all_vkt_as_dict(clipped_test_v_travel_events, network_gdf)

({'freight_carrier_0_veh_carrier0_Van_1_2': 54113.18740448947},
 {'freight_carrier_0_veh_carrier0_Van_1_2': 16269.0})

In [20]:
test_v_events['type'].unique()

array(['actend', 'departure', 'PersonEntersVehicle',
       'vehicle enters traffic', 'vehicle leaves traffic',
       'PersonLeavesVehicle', 'arrival', 'actstart',
       'Freight tour starts', 'coldEmissionEvent',
       'Freight shipment pickup starts', 'Freight shipment pickup ends',
       'left link', 'entered link', 'warmEmissionEvent',
       'Freight shipment delivered starts',
       'Freight shipment delivered ends', 'Freight tour ends'],
      dtype=object)

# Shipment count

In [65]:
shipments_df = shipment_anls.read_shipment_events(root_path + scenario_kw)
shipments_df

,capacityDemand,pickupDuration,shipmentId,carrierId,type,link,vehicle,time
0,9.0,0.0,shipment_195,carrier_0,Freight shipment pickup starts,333784188_r_3,freight_carrier_0_veh_carrier0_Van_1_1,0.0
1,14.0,0.0,shipment_61,carrier_0,Freight shipment pickup starts,333784188_r_3,freight_carrier_0_veh_carrier0_Van_1_2,0.0
2,16.0,0.0,shipment_129,carrier_0,Freight shipment pickup starts,333784188_r_3,freight_carrier_0_veh_carrier0_Van_1_2,1.0
3,18.0,0.0,shipment_5,carrier_0,Freight shipment pickup starts,333784188_r_3,freight_carrier_0_veh_carrier0_Van_1_1,1.0
4,13.0,0.0,shipment_183,carrier_0,Freight shipment pickup starts,333784188_r_3,freight_carrier_0_veh_carrier0_Van_1_1,2.0
...,...,...,...,...,...,...,...,...
195,24.0,0.0,shipment_142,carrier_0,Freight shipment pickup starts,27566523_11,freight_carrier_0_veh_carrier0_Van_1_1,29774.0
196,15.0,0.0,shipment_112,carrier_0,Freight shipment pickup starts,27566523_11,freight_carrier_0_veh_carrier0_Van_1_1,29775.0
197,18.0,0.0,shipment_109,carrier_0,Freight shipment pickup starts,27566523_11,freight_carrier_0_veh_carrier0_Van_1_1,29776.0
198,6.0,0.0,shipment_47,carrier_0,Freight shipment pickup starts,27566523_11,freight_carrier_0_veh_carrier0_Van_1_1,29777.0
